In [30]:
import graphlab as gl
business = gl.SFrame.read_json('./../intermediate_data/split_sentences_business_final.json', orient='lines')

Finished parsing file /Users/kumanik/Desktop/ASU_Work/ML/intermediate_data/split_sentences_business_final.json

Parsing completed. Parsed 100 lines in 0.654584 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1847 lines. Lines per second: 1353.18

Read 11695 lines. Lines per second: 1819

Read 21041 lines. Lines per second: 1740.71

Read 30744 lines. Lines per second: 1781.23

Finished parsing file /Users/kumanik/Desktop/ASU_Work/ML/intermediate_data/split_sentences_business_final.json

Parsing completed. Parsed 36263 lines in 20.0917 secs.

In [31]:
business_tmp_1 = business[100:300]
business_tmp_2 = business[300:500]
business_tmp_3 = business[500:700]
business_tmp_4 = business[700:900]
business_tmp_5 = business[900:1000]


business_tmp_1.export_json('100to300.json', orient='lines')
business_tmp_2.export_json('300to500.json', orient='lines')
business_tmp_3.export_json('500to700.json', orient='lines')
business_tmp_4.export_json('700to900.json', orient='lines')
business_tmp_5.export_json('900to1000.json', orient='lines')



In [ ]:
import json
from textblob import Word
from gensim.models import Word2Vec as wv
model = wv.load('./../word2vecModel/10_min_model_w2vec_entire_data_punc_stop_removed')
with open('./../word2vecModel/all_bow.json', "r") as file_read:
    bag_of_words = json.load(file_read)

def lemmatize(list_of_words):
    lemmatized_words = []
    for word in list_of_words:
        print word
        lemmatized_words.append(Word(word).lemmatize())
    return lemmatized_words
    
food_bag_of_words = lemmatize(bag_of_words['food'])
service_bag_of_words = lemmatize(bag_of_words['service'])
discount_bag_of_words = lemmatize(bag_of_words['discount'])
ambience_bag_of_words = lemmatize(bag_of_words['ambience'])

def remove_unpresent_words(list_of_words):
    for word in list_of_words:
        if not word in model.vocab:
            list_of_words.remove(word)
remove_unpresent_words(food_bag_of_words)
remove_unpresent_words(service_bag_of_words)
remove_unpresent_words(discount_bag_of_words)
remove_unpresent_words(ambience_bag_of_words)
            

def extract_parts_of_speech(sentence):
    parts = {}
    adj = gl.text_analytics.extract_parts_of_speech(sentence,chosen_pos=[gl.text_analytics.PartOfSpeech.ADJ])
    dict_of_adj = adj[0]['ADJ']
    parts['Adjectives'] = dict_of_adj.keys()
    noun = gl.text_analytics.extract_parts_of_speech(sentence,chosen_pos=[gl.text_analytics.PartOfSpeech.NOUN])
    dict_of_noun = noun[0]['NOUN']
    parts['Nouns'] = dict_of_noun.keys()
    adverb = gl.text_analytics.extract_parts_of_speech(sentence,chosen_pos=[gl.text_analytics.PartOfSpeech.ADV])
    dict_of_adverb = adverb[0]['ADV']
    parts['Adverbs'] = dict_of_adverb.keys()
    return parts

def find_similarity_with_food(list_of_words):
    if len(list_of_words) is 0:
        return 0
    similarity = model.n_similarity(list_of_words, food_bag_of_words)
    return similarity
    
def find_similarity_with_service(list_of_words):
    if len(list_of_words) is 0:
            return 0
    return model.n_similarity(list_of_words, service_bag_of_words)
   
def find_similarity_with_discount(list_of_words):
    if len(list_of_words) is 0:
        return 0
    return model.n_similarity(list_of_words, discount_bag_of_words)
    

def find_similarity_with_ambience(list_of_words):
    if len(list_of_words) is 0:
        return 0
    return model.n_similarity(list_of_words, ambience_bag_of_words)

def expand_word_list(list_of_words):
    expanded_word_list = []  
    if len(list_of_words) is 0:
        return expanded_word_list
    for word in list_of_words:
        if word in model.vocab:
            word2vecList = model.most_similar(word, topn=5)
            for word_tuple in word2vecList:
                expanded_word_list.append(word_tuple[0]) 
    return expanded_word_list
    
count = 1
def classify_sentences(list_of_sentences):
    print count 
    food_sentences = []
    service_sentences = []
    discount_sentences = []
    ambience_sentences = []
    for j in range(len(list_of_sentences)):
        sa = gl.SArray([list_of_sentences[j]])
        parts_of_speech = extract_parts_of_speech(sa)
        
        
        
        expanded_nouns = expand_word_list(lemmatize(parts_of_speech['Nouns']))
        expanded_adj = expand_word_list(lemmatize(parts_of_speech['Adjectives']))
        expanded_adverb = expand_word_list(lemmatize(parts_of_speech['Adverbs']))
    
        food_similarity = find_similarity_with_food(expanded_nouns)
        service_similarity = find_similarity_with_service(expanded_nouns+expanded_adj+expanded_adverb)
        discount_similarity = find_similarity_with_discount(expanded_nouns+expanded_adj+expanded_adverb)
        ambience_similarity = find_similarity_with_ambience(expanded_nouns+expanded_adj+expanded_adverb)
        
        threshold = 0.5;
        
        if food_similarity >= threshold:
            food_sentences.append(list_of_sentences[j])
        if service_similarity >= threshold:
            service_sentences.append(list_of_sentences[j])
        if discount_similarity >= threshold:
            discount_sentences.append(list_of_sentences[j])
        if ambience_similarity >= threshold:
            ambience_sentences.append(list_of_sentences[j])
        
    classified_sentences = {}
    classified_sentences['food_sentences'] = food_sentences
    classified_sentences['service_sentences'] = service_sentences
    classified_sentences['discount_sentences'] = discount_sentences
    classified_sentences['ambience_sentences'] = ambience_sentences
    count += 1
    return classified_sentences
    
business_tmp['classified_sentences'] = business_tmp['splitSentences'].apply(lambda x: classify_sentences(x))



        

In [ ]:
business_tmp.head()